In [6]:
import os
import torch
import torchvision.transforms as transforms
from torchvision import datasets, models
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

# Set device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Define data directories
data_dir = 'pif'  #dataset path
train_dir = os.path.join(data_dir, 'train')
val_dir = os.path.join(data_dir, 'val')

# Define transformations
data_transforms = {
    'train': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
    'val': transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ]),
}

# Load datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
                  for x in ['train', 'val']}
dataloaders = {x: DataLoader(image_datasets[x], batch_size=32, shuffle=True, num_workers=4)
                for x in ['train', 'val']}

num_classes = len(image_datasets['train'].classes)

model = models.densenet121(pretrained=True)

model.classifier = nn.Linear(model.classifier.in_features, num_classes)

model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, patience=5):
    best_val_loss = float('inf')
    epochs_without_improvement = 0

    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()   

            running_loss = 0.0
            running_corrects = 0

            
            for inputs, labels in dataloaders[phase]:
                inputs, labels = inputs.to(device), labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs, labels)
                    _, preds = torch.max(outputs, 1)

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(image_datasets[phase])
            epoch_acc = running_corrects.double() / len(image_datasets[phase])

            print(f'{phase} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            if phase == 'val':
                if epoch_loss < best_val_loss:
                    best_val_loss = epoch_loss
                    epochs_without_improvement = 0  
                    print("Validation loss improved, saving model...")
                    torch.save(model.state_dict(), 'best_model_dense.pt')  
                else:
                    epochs_without_improvement += 1
                    if epochs_without_improvement >= patience:
                        print(f"Early stopping triggered after {patience} epochs without improvement.")
                        return model  

    return model


model = train_model(model, dataloaders, criterion, optimizer, num_epochs=20, patience=5)


Epoch 1/20
----------
train Loss: 0.6940 Acc: 0.7614
val Loss: 1.4481 Acc: 0.5467
Validation loss improved, saving model...
Epoch 2/20
----------
train Loss: 0.4264 Acc: 0.8505
val Loss: 1.0989 Acc: 0.7000
Validation loss improved, saving model...
Epoch 3/20
----------
train Loss: 0.3519 Acc: 0.8767
val Loss: 1.1515 Acc: 0.6711
Epoch 4/20
----------
train Loss: 0.3069 Acc: 0.8921
val Loss: 0.6861 Acc: 0.7911
Validation loss improved, saving model...
Epoch 5/20
----------
train Loss: 0.2824 Acc: 0.8994
val Loss: 1.6684 Acc: 0.7200
Epoch 6/20
----------
train Loss: 0.2591 Acc: 0.9083
val Loss: 1.3101 Acc: 0.7578
Epoch 7/20
----------
train Loss: 0.2421 Acc: 0.9135
val Loss: 1.6275 Acc: 0.7200
Epoch 8/20
----------
train Loss: 0.2261 Acc: 0.9205
val Loss: 1.7269 Acc: 0.7711
Epoch 9/20
----------
train Loss: 0.2008 Acc: 0.9288
val Loss: 1.5049 Acc: 0.7511
Early stopping triggered after 5 epochs without improvement.
